In [19]:
import pandas as pd
import numpy as np
import re

In [20]:
arts = pd.read_csv('../data/epir_train/articles.csv')
arts.head()

,Unnamed: 0,id,sys_lang,projects,title,content,url
0,2884,135747,ru,departament-kkbtu-sko,Бешенство – смертельная угроза для человека!,"Бешенство – природно-очаговое особо опасное, с...",https://www.gov.kz/memleket/entities/departame...
1,2885,119917,ru,kgd-karaganda,Порядок администрирования налога на транспортн...,В соответствии с пунктом 2 статьи 66 Закона «О...,https://www.gov.kz/memleket/entities/kgd-karag...
2,2886,8414,ru,turkestan-tolebi-audany,Спорт,Миссия: формирование единой государственной по...,https://www.gov.kz/memleket/entities/turkestan...
3,2887,118430,kk,qazalem,ҚАЗАҚСТАННЫҢ МЕМЛЕКЕТТІК ОРГАНДАРЫМЕН БАЙЛАНЫС,Электрондық үкіметтің мүмкіндіктерін толық пай...,https://www.gov.kz/memleket/entities/qazalem/p...
4,2888,5319,kk,kyzylorda-kzo,2022 жылға жергілікті атқарушы органдармен сыб...,Қызылорда қаласының әкімдігі р/с Іс-шаралар 20...,https://www.gov.kz/memleket/entities/kyzylorda...


In [3]:
arts = arts.fillna('')

In [4]:
arts['text'] = arts['title'] + '. ' + arts['content']

In [5]:
arts['text'] = arts['text'].str.strip()

In [18]:
arts[arts['text'].apply(len) == 9]

,Unnamed: 0,id,sys_lang,projects,title,content,url,text
525,3409,122988,en,kostanai-bmalin-audany-akimat,Salyqbot,,https://www.gov.kz/memleket/entities/kostanai-...,Salyqbot.
777,3661,50128,qq,sko-petropavl,Anonstar,,https://www.gov.kz/memleket/entities/sko-petro...,Anonstar.
1575,4459,70771,ru,aqmola-sand,COVID-19,,https://www.gov.kz/memleket/entities/aqmola-sa...,COVID-19.
2468,5352,617,ru,enbek,Миграция,,https://www.gov.kz/memleket/entities/enbek/pre...,Миграция.
3041,5925,59052,en,vko-beskaragay,Маслихат,,https://www.gov.kz/memleket/entities/vko-beska...,Маслихат.
...,...,...,...,...,...,...,...,...
81341,84225,124585,kk,karaganda-nurinsk,Шешімдер,,https://www.gov.kz/memleket/entities/karaganda...,Шешімдер.
82635,85519,54931,qq,vko-altai,Ár túrli,,https://www.gov.kz/memleket/entities/vko-altai...,Ár túrli.
96322,99206,113730,kk,kyzylorda,7 kun.kz,,https://www.gov.kz/memleket/entities/kyzylorda...,7 kun.kz.
96942,99826,124586,kk,karaganda-nurinsk,Хабарлар,,https://www.gov.kz/memleket/entities/karaganda...,Хабарлар.


In [66]:
arts = arts[arts['text'].apply(len) > 0]

In [67]:
arts['text'] = arts['text'].str.lower()

In [79]:
arts['text'] = arts['text'].apply(lambda t: re.sub(r'[^\w\s]', '', t))

In [83]:
arts['text'].iloc[1]

'порядок администрирования налога на транспортные средства физических лиц в соответствии с пунктом 2 статьи 66 закона о дорожном движении обязанность зарегистрировать транспортные средства в уполномоченном органе по обеспечению безопасности дорожного движения возложена на владельцев транспортных средств в то же время прекращение государственной регистрации транспортного средства производится по заявлению владельца в связи с утратой отчуждением транспортного средства другому лицу а также снятием его с регистрационного учета перед утилизацией или вывозом на постоянное пребывание за пределы республики казахстан или в соответствии с иными обстоятельствами таким образом обязанность регистрации и прекращения регистрации транспортного средства в уполномоченном органе а именно в департаменте полиции карагандинской области возложена на владельцев автотранспортных средств уплата налога физическими лицами производится в бюджет по месту жительства не позднее 1 апреля года следующего за отчетным на

In [ ]:
arts = arts[['Unnamed: 0', 'text', 'url']]

In [88]:
arts.to_csv('articles_cleaned.csv', index=False)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import pandas as pd

# Загрузка обучающего и тестового наборов данных
train_data = pd.read_csv("train_dataset.csv")
test_data = pd.read_csv("test_dataset.csv")

# Создание TF-IDF векторизатора
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(train_data["text"])

# Функция для получения ответа на вопрос
def get_answer(question):
    question_vector = tfidf_vectorizer.transform([question])
    cosine_similarities = linear_kernel(question_vector, tfidf_matrix)
    most_similar_index = cosine_similarities.argmax()
    corresponding_unnamed = train_data.iloc[most_similar_index]["Unnamed: 0"]
    return corresponding_unnamed

# Проход по вопросам в тестовом наборе и получение соответствующего "Unnamed: 0"
corresponding_unnamed_list = []
for question in test_data["question"]:
    corresponding_unnamed = get_answer(question)
    corresponding_unnamed_list.append(corresponding_unnamed)

# Добавление "Unnamed: 0" к тестовому набору данных
test_data["Unnamed: 0"] = corresponding_unnamed_list

# Сохранение результата в файл
test_data.to_csv("test_dataset_with_unnamed.csv", index=False)